In [2]:
library(data.table)
library(qvalue)

In [3]:
## CD (Crohn's disease) SMR (summarised Mendelian Randomisation) results

In [4]:
# Matrix eQTL results
matrix_eqtl_smr_results_dir = "/directflow/SCCGGroupShare/projects/angxue/proj/SAIGE-eQTL/SMR/"
matrix_ibd_file = paste0(matrix_eqtl_smr_results_dir, "cd_all_14_celltypes_all_smr.txt")
matrix_df = fread(matrix_ibd_file)
head(matrix_df,2)

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,p_GWAS,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
FCRL5,1,FCRL5,157483167,rs12741984,1,157534818,G,A,0.787234,⋯,0.2133,0.113780,0.00486738,7.500497e-121,0.185382,0.148752,0.2126742,0.4619789,20,BimmNaive
RPS8,1,RPS8,45240923,rs12120833,1,45241285,C,G,0.850097,⋯,0.4571,-0.105432,0.00500701,1.980368e-98,0.147458,0.198581,0.4577494,0.3150508,20,BimmNaive


In [5]:
# Calculate number of unique genes, and combination of genes + celltypes
length(unique(matrix_df$Gene))
matrix_df$comb = paste0(matrix_df$Gene,"_",matrix_df$Cell_type)
length(unique(matrix_df$comb))
head(matrix_df,2)

[1] 6091

[1] 14518

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
FCRL5,1,FCRL5,157483167,rs12741984,1,157534818,G,A,0.787234,⋯,0.113780,0.00486738,7.500497e-121,0.185382,0.148752,0.2126742,0.4619789,20,BimmNaive,FCRL5_BimmNaive
RPS8,1,RPS8,45240923,rs12120833,1,45241285,C,G,0.850097,⋯,-0.105432,0.00500701,1.980368e-98,0.147458,0.198581,0.4577494,0.3150508,20,BimmNaive,RPS8_BimmNaive


In [6]:
nrow(matrix_df[matrix_df$p_SMR < 0.05/length(unique(matrix_df$Gene)),])
nrow(matrix_df[matrix_df$p_SMR < 0.05/length(unique(matrix_df$comb)),])

[1] 45

[1] 38

In [7]:
# SAIGE-QTL results
saige_eqtl_smr_results_dir = "/directflow/SCCGGroupShare/projects/angxue/proj/SAIGE-eQTL/SMR/saige_eQTL/"
saige_ibd_file = paste0(saige_eqtl_smr_results_dir, "cd_all_14_celltypes_all_smr.txt")
saige_df = fread(saige_ibd_file)
head(saige_df,2)

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,p_GWAS,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
NOC2L,1,NOC2L,879584,rs3748595,1,887560,A,C,0.0672147,⋯,0.5676,-0.153020,0.0352961,1.455500e-05,-0.073327,0.129544,0.5713679,0.1533406,3,BimmNaive
C1orf86,1,C1orf86,2115903,rs11587831,1,2110848,T,G,0.7853000,⋯,0.3485,-0.115809,0.0148762,6.980209e-15,0.134616,0.144356,0.3510651,0.6961858,8,BimmNaive


In [8]:
length(unique(saige_df$Gene))
saige_df$comb = paste0(saige_df$Gene,"_",saige_df$Cell_type)
length(unique(saige_df$comb))
head(saige_df,2)

[1] 5671

[1] 16398

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,b_eQTL,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
NOC2L,1,NOC2L,879584,rs3748595,1,887560,A,C,0.0672147,⋯,-0.153020,0.0352961,1.455500e-05,-0.073327,0.129544,0.5713679,0.1533406,3,BimmNaive,NOC2L_BimmNaive
C1orf86,1,C1orf86,2115903,rs11587831,1,2110848,T,G,0.7853000,⋯,-0.115809,0.0148762,6.980209e-15,0.134616,0.144356,0.3510651,0.6961858,8,BimmNaive,C1orf86_BimmNaive


In [9]:
# significant results at p<0.05/M with M either # genes or # gene-celltype combinations
nrow(saige_df[saige_df$p_SMR < 0.05/length(unique(saige_df$Gene)),])
nrow(saige_df[saige_df$p_SMR < 0.05/length(unique(saige_df$comb)),])

[1] 46

[1] 43

In [10]:
### p < 0.05/M with M-number of unique gene-celltype combinations tested by both methods

In [11]:
matrix_combs = unique(matrix_df$comb)
saige_combs = unique(saige_df$comb)
common_combs = saige_combs[saige_combs %in% matrix_combs]
length(common_combs)

[1] 11985

In [12]:
matrix_df_common = matrix_df[matrix_df$comb %in% common_combs,]
nrow(matrix_df_common)
nrow(matrix_df_common[matrix_df_common$p_SMR < (0.05/length(common_combs)),])

[1] 11985

[1] 39

In [13]:
saige_df_common = saige_df[saige_df$comb %in% common_combs,]
nrow(saige_df_common)
nrow(saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),])

[1] 11985

[1] 43

In [14]:
# significant results at FDR<5% instead (using qvalue)
matrix_df_common$qv = qvalue(matrix_df_common$p_SMR)$qvalues
nrow(matrix_df_common[matrix_df_common$qv<0.05,])

[1] 138

In [15]:
saige_df_common$qv = qvalue(saige_df_common$p_SMR)$qvalues
nrow(saige_df_common[saige_df_common$qv<0.05,])

[1] 162

In [17]:
# Using p < (0.05 / n_tests, gene-celltype comb)

matrix_sign_combs = matrix_df_common[matrix_df_common$p_SMR < (0.05/length(common_combs)),"comb"]$comb
length(matrix_sign_combs)
head(matrix_sign_combs)

saige_sign_combs = saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),"comb"]$comb
length(saige_sign_combs)
head(saige_sign_combs)

length(saige_sign_combs[saige_sign_combs %in% matrix_sign_combs])

[1] 39

[1] "ERAP2_BimmNaive"  "TUFM_BimmNaive"   "ORMDL3_BimmNaive" "ERAP2_Bmem"      
[5] "TUFM_Bmem"        "ORMDL3_Bmem"

[1] 43

[1] "ERAP2_BimmNaive"  "TUFM_BimmNaive"   "ORMDL3_BimmNaive" "ERAP2_Bmem"      
[5] "TUFM_Bmem"        "ORMDL3_Bmem"

[1] 37

In [18]:
# Using FDR < 0.05

matrix_sign_combs = matrix_df_common[matrix_df_common$qv < 0.05,"comb"]$comb
length(matrix_sign_combs)

saige_sign_combs = saige_df_common[saige_df_common$qv < 0.05,"comb"]$comb
length(saige_sign_combs)

length(saige_sign_combs[saige_sign_combs %in% matrix_sign_combs])

[1] 138

[1] 162

[1] 125

In [16]:
## SAIGE-QTL SMR gene list
saige_df_sign = saige_df_common[saige_df_common$p_SMR < (0.05/length(common_combs)),]
saige_df_sign

probeID,ProbeChr,Gene,Probe_bp,topSNP,topSNP_chr,topSNP_bp,A1,A2,Freq,⋯,se_eQTL,p_eQTL,b_SMR,se_SMR,p_SMR,p_HEIDI,nsnp_HEIDI,Cell_type,comb,qv
<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>,<dbl>
ERAP2,5,ERAP2,96211643,rs27295,5,96358687,C,T,0.431335,⋯,0.02186180,9.580355e-47,-0.292410,0.0491637,2.719701e-09,7.152581e-01,20,BimmNaive,ERAP2_BimmNaive,2.170082e-06
TUFM,16,TUFM,28853732,rs7187776,16,28857645,A,G,0.615571,⋯,0.01473370,3.420663e-67,-0.333696,0.0590836,1.624574e-08,7.104661e-02,20,BimmNaive,TUFM_BimmNaive,7.636231e-06
ORMDL3,17,ORMDL3,38077294,rs8067378,17,38051348,A,G,0.491779,⋯,0.02442030,1.576034e-50,0.199113,0.0401591,7.118453e-07,1.339933e-01,20,BimmNaive,ORMDL3_BimmNaive,1.849102e-04
ERAP2,5,ERAP2,96211643,rs27998,5,96367408,G,A,0.428917,⋯,0.02443750,7.455040e-54,-0.238227,0.0402680,3.297681e-09,8.812517e-01,20,Bmem,ERAP2_Bmem,2.170082e-06
TUFM,16,TUFM,28853732,rs7187776,16,28857645,A,G,0.615571,⋯,0.01434550,3.904825e-45,-0.421010,0.0765345,3.778108e-08,1.662807e-01,20,Bmem,TUFM_Bmem,1.553896e-05
ORMDL3,17,ORMDL3,38077294,rs7359623,17,38049589,C,T,0.496132,⋯,0.02815960,1.782539e-39,0.204092,0.0404580,4.545983e-07,4.388768e-02,20,Bmem,ORMDL3_Bmem,1.319799e-04
SCAMP3,1,SCAMP3,155225770,rs1076556,1,155234821,A,G,0.713250,⋯,0.01138940,1.048093e-32,-0.641302,0.1306390,9.155943e-07,1.544640e-01,20,CD4all,SCAMP3_CD4all,2.259446e-04
GPX1,3,GPX1,49394609,rs1801143,3,49570200,C,T,0.701161,⋯,0.00677544,7.627006e-13,2.124130,0.4262650,6.256809e-07,6.638163e-01,20,CD4all,GPX1_CD4all,1.715116e-04
ERAP2,5,ERAP2,96211643,rs2910686,5,96252589,T,C,0.567215,⋯,0.01822860,4.685026e-155,-0.189532,0.0298614,2.194715e-10,3.353872e-01,20,CD4all,ERAP2_CD4all,5.359490e-07
